In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1.2 인사 데이터로 직원 퇴사 예측하기

In [ ]:
!pip install pandas-profiling==3.1.0

#!pip install --upgrade pandas-profiling
#!pip install markupsafe==2.0.1

### 상위 런타임 메뉴에서 "런타임 다시 시작"을 선택한 후 아래 코드를 실행합니다.

WARNING: The following packages were previously imported in this runtime:
  [markupsafe,pandas_profiling,requests]
You must restart the runtime in order to use newly installed versions.

## 판다스 프로파일로 EDA하기

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

In [6]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/HR Analytics_ Job Change of Data Scientists/aug_train.csv')

In [7]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(train, test_size=0.2, random_state=0)
train, val = train_test_split(train, test_size=0.2, random_state=0)

print('train & validation Dataset : ', train.shape, val.shape)
print('test Dataset : ', test.shape)

train & validation Dataset :  (12260, 14) (3066, 14)
test Dataset :  (3832, 14)


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12260 entries, 6820 to 16553
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             12260 non-null  int64  
 1   city                    12260 non-null  object 
 2   city_development_index  12260 non-null  float64
 3   gender                  9418 non-null   object 
 4   relevent_experience     12260 non-null  object 
 5   enrolled_university     12010 non-null  object 
 6   education_level         11971 non-null  object 
 7   major_discipline        10491 non-null  object 
 8   experience              12217 non-null  object 
 9   company_size            8509 non-null   object 
 10  company_type            8367 non-null   object 
 11  last_new_job            12000 non-null  object 
 12  training_hours          12260 non-null  int64  
 13  target                  12260 non-null  float64
dtypes: float64(2), int64(2), object(10)

In [9]:
import pandas_profiling

profile = train.profile_report()
profile

Output hidden; open in https://colab.research.google.com to view.

## 1.2.3 feature_column로 데이터 전처리하기

In [10]:
from tensorflow import feature_column

feature_columns = []

# 수치형 피처
for numeric_col in ['enrollee_id', 'city_development_index', 'training_hours']:
  feature_columns.append(feature_column.numeric_column(numeric_col))

In [11]:
train['gender'].value_counts()

Male      8496
Female     808
Other      114
Name: gender, dtype: int64

In [12]:
## 범주형 피처
gender = feature_column.categorical_column_with_vocabulary_list(
      'gender', ['Male', 'Female', 'Other'])
gender_one_hot = feature_column.indicator_column(gender)
feature_columns.append(gender_one_hot)

relevent_experience = feature_column.categorical_column_with_vocabulary_list(
      'relevent_experience', ['Has relevent experience', 'No relevent experience'])
relevent_experience_one_hot = feature_column.indicator_column(relevent_experience)
feature_columns.append(relevent_experience_one_hot)

enrolled_university = feature_column.categorical_column_with_vocabulary_list(
      'enrolled_university', ['no_enrollment', 'Full time course', 'Part time course'])
enrolled_university_one_hot = feature_column.indicator_column(enrolled_university)
feature_columns.append(enrolled_university_one_hot)

In [13]:
## 임베딩 피처
education_level = feature_column.categorical_column_with_vocabulary_list(
      'education_level', ['Graduate', 'High School', 'Masters', 'Primary School', 'Phd'])
education_level_embedding = feature_column.embedding_column(education_level, dimension=10)

feature_columns.append(education_level_embedding)

# 임베딩 피처
company_size = feature_column.categorical_column_with_vocabulary_list(
      'company_size', ['50-99', '100-500', '10000+', '10/49', '1000-4999', '<10', '500-999', '5000-9999'])
company_size_embedding = feature_column.embedding_column(company_size, dimension=10)

feature_columns.append(company_size_embedding)

In [14]:
train = train.dropna()
val = val.dropna()
test = test.dropna()

In [15]:
def dataframe_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('target')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [16]:
batch_size = 32

train_ds = dataframe_to_dataset(train, batch_size=batch_size)
val_ds = dataframe_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = dataframe_to_dataset(test, shuffle=False, batch_size=batch_size)

## 1.2.4 모델 빌드 및 컴파일

In [17]:
from tensorflow.keras.layers import DenseFeatures

DenseFeatures_Layer = DenseFeatures(feature_columns)

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
  DenseFeatures_Layer,
  Dense(128, activation='relu'),
  Dense(64, activation='relu'),
  Dense(32, activation='relu'),
  Dense(1, activation='sigmoid')
])

In [19]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## 1.2.5 콜백을 사용하여 훈련 주기 마지막을 트리거

In [20]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(patience = 10)
checkpoint_model = ModelCheckpoint('ModelCheckpoint.h5')

In [24]:
history = model.fit(train_ds,
          validation_data=val_ds,
          epochs=100,
          callbacks=[early_stopping, checkpoint_model])

Epoch 1/100
182/182 [==============================] - 1s 5ms/step - loss: 3.2328 - accuracy: 0.7344 - val_loss: 7.0083 - val_accuracy: 0.8263
Epoch 2/100
182/182 [==============================] - 1s 4ms/step - loss: 2.7153 - accuracy: 0.7404 - val_loss: 7.1922 - val_accuracy: 0.8263
Epoch 3/100
182/182 [==============================] - 1s 4ms/step - loss: 1.9705 - accuracy: 0.7506 - val_loss: 1.2298 - val_accuracy: 0.8263
Epoch 4/100
182/182 [==============================] - 1s 5ms/step - loss: 1.6323 - accuracy: 0.7444 - val_loss: 1.0839 - val_accuracy: 0.8263
Epoch 5/100
182/182 [==============================] - 1s 5ms/step - loss: 1.3941 - accuracy: 0.7413 - val_loss: 1.4081 - val_accuracy: 0.8263
Epoch 6/100
182/182 [==============================] - 1s 4ms/step - loss: 1.3300 - accuracy: 0.7619 - val_loss: 0.4831 - val_accuracy: 0.8263
Epoch 7/100
182/182 [==============================] - 1s 4ms/step - loss: 1.7603 - accuracy: 0.7500 - val_loss: 0.5377 - val_accuracy: 0.8263

In [25]:
loss, accuracy = model.evaluate(test_ds)
print("accuracy: ", accuracy)

56/56 [==============================] - 0s 3ms/step - loss: 0.4932 - accuracy: 0.8336
accuracy:  0.8336172699928284


## 1.2.6 훈련된 모델의 손실 및 정확도 시각화

In [26]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(specs=[[{"secondary_y": True}]])

# model history
epoch = history.epoch

loss = history.history['loss']
val_loss = history.history['val_loss']
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Scatter
fig.add_trace(go.Scatter(x=epoch, y=loss, name="loss"),secondary_y=False,)
fig.add_trace(go.Scatter(x=epoch, y=val_loss, name="val_loss"),secondary_y=False,)
fig.add_trace(go.Scatter(x=epoch, y=accuracy, name="accuracy"),secondary_y=True,)
fig.add_trace(go.Scatter(x=epoch, y=val_accuracy, name="val_accuracy"),secondary_y=True,)

# Templates configuration, Default template: 'plotly'
# Available templates: ['ggplot2', 'seaborn', 'simple_white', 'plotly','plotly_white', 'plotly_dark', 'presentation', 'xgridoff','ygridoff', 'gridon', 'none']
fig.update_layout(title_text="<b>Loss/Accuracy of Model</b>", template='plotly')

fig.update_xaxes(title_text="Epoch")
fig.update_yaxes(title_text="Loss", secondary_y=False)
fig.update_yaxes(title_text="Accuracy", secondary_y=True)

fig.show()